In [ ]:
from pdfminer import high_level
from PIL import Image
import fitz
import io
from fpdf import FPDF

In [ ]:
#extract holder details and split them in a list
path = r'C:\Users\Yatharth\Documents\VacCert-Project\certs\jv.pdf'
text = high_level.extract_text(path)
print(text)
ls = text.split("\n\n")
ls = ls[:11]
print(ls)
#dictionary for storing holder information
info = ["name", "age", "sex", "verif", "uhid", "refID", "vax", "dose1", "dose2"]
info_dict = {}

for i in range(len(info)):
    info_dict[info[i]] = ls[i]
print(info_dict)
if info_dict["verif"].split()[0] != "Aadhaar":
    info_dict["uhid"] = ''
    for i in range(5,len(info)):
        info_dict[info[i]] = ls[i-1]
print(info_dict)


In [ ]:
#Open PDF again, get the QR code and save for use
pdf_file = fitz.open(path)
page = pdf_file[0]
image_list = page.getImageList()
xref = image_list[0][0]
base_image = pdf_file.extractImage(xref)
image_bytes = base_image["image"]
image_ext = base_image["ext"]
qr = Image.open(io.BytesIO(image_bytes))
qr.save(open(f"qr.{image_ext}", "wb"))

In [ ]:
qr_link = r'C:\Users\Yatharth\Documents\VacCert-Project\vaccine-certificate-shortener\qr.png'
hfw_img = r'C:\Users\Yatharth\Documents\VacCert-Project\vaccine-certificate-shortener\imgfiles\mohfw.png'

In [ ]:
#Creating new certificate with PyFPDF
ph_w = 281.25
ph_h = 500.25
ln_y = 274
ln_x1 = 21
ln_x2 = 260
def mob_1():
    pdf = FPDF('P','pt', (ph_w,ph_h))
    pdf.add_page()
    pdf.set_margins(left=0, top=0, right=0)
    pdf.image(hfw_img,x=75,y=24.5,w=132,h=68,type="png")
    pdf.set_font('Arial', 'B', 11.2)
    pdf.set_text_color(0,0,125)
    pdf.cell(w=225,h=166,txt="Final COVID-19 Vaccine Certificate",border=0,align='C',ln=1)
    pdf.set_line_width(0.1)
    pdf.line(ln_x1,ln_y,ln_x2,ln_y)
    pdf.image(qr_link,x=49,y=302,w=184,h=184,type="png")
    pdf.set_font('Arial', size=7)
    pdf.set_text_color(0,0,0) 
    pdf.ln(h=96)
    pdf.cell(w=282,txt="https://verify.cowin.gov.in",border=0,align='C',link="https://verify.cowin.gov.in",ln=2)
    return pdf

In [ ]:
mob = mob_1()
mob.set_font('Arial', 'B',size=9)
mob.set_text_color(31,73,125)
info = "Vaccine \nBeneficiary Name \nAge \nGender \nVerified ID \nUHID \nReference ID \nVaccination Dates"
seq = ['vax','name','age','sex','verif','uhid','refID','dose1','dose2']
data = ''
for i in seq:
    data = data + info_dict[i] + "\n"
mob.set_xy(x = 18,y = 127)
mob.multi_cell(w=250,h=15,txt=info,align='L',border=0)
mob.set_xy(x = 18,y = 127)
mob.set_font('Arial',size=9)
mob.set_text_color(0,0,0)
mob.multi_cell(w=242,h=15,txt=data,align='R',border=0)
mob.output('mobile1.pdf', 'F')
